In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# =========================
# 0) CÀI THƯ VIỆN
# =========================
!pip -q install --upgrade pip
!pip -q install "sentence-transformers>=3.0.0" faiss-cpu "rank_bm25>=0.2.2" datasets
!pip -q install "transformers>=4.41.0" "accelerate>=0.30.0"
!pip -q install fastapi uvicorn nest_asyncio
!pip install ijson

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 68.1 MB/s eta 0:00:00


In [4]:
# =========================
# 1) CẤU HÌNH & IMPORT
# =========================
import os, json, math, gzip, pickle, textwrap, re, uuid, time
from typing import List, Dict, Tuple, Any

import numpy as np
import faiss

from sentence_transformers import SentenceTransformer, CrossEncoder
from rank_bm25 import BM25Okapi

# --- Đường dẫn làm việc trên Colab ---
WORK_DIR = "/content/drive/MyDrive/eGov-Bot/final_RAG"
os.makedirs(WORK_DIR, exist_ok=True)

RAW_JSON_PATH   = f"{WORK_DIR}/toan_bo_du_lieu_final.json"      # dữ liệu gốc (list các thủ tục)
CHUNKS_JSONL    = f"{WORK_DIR}/chunks.jsonl"  # dữ liệu đã chunk (mỗi dòng 1 chunk)
FAISS_INDEX     = f"{WORK_DIR}/index.faiss"   # file chỉ mục FAISS
METAS_PKL_GZ    = f"{WORK_DIR}/metas.pkl.gz"  # metadata kèm text
BM25_PICKLE     = f"{WORK_DIR}/bm25.pkl.gz"   # bm25 corpus (tuỳ chọn)

# --- Model embedding & reranker ---
EMBED_MODEL_NAME   = "AITeamVN/Vietnamese_Embedding"



In [ ]:
# =========================
# 3) HÀM TIỀN XỬ LÝ & CHUNKING (ít RAM) - với semantic chunking
# =========================
import json
import re
import uuid
import gc
import os
import sys
from typing import Generator, Dict, Any, List

# Thêm cho semantic chunking
import math
import numpy as np


# Ép NumPy/BLAS chỉ dùng 2 threads
os.environ["OMP_NUM_THREADS"] = "2"
os.environ["MKL_NUM_THREADS"] = "2"
os.environ["OPENBLAS_NUM_THREADS"] = "2"

# Nếu có torch thì ép dùng 2 threads
try:
    import torch
    torch.set_num_threads(2)
except ImportError:
    torch = None

FIELD_ORDER = [
    "ten_thu_tuc", "yeu_cau_dieu_kien", "thanh_phan_ho_so",
    "trinh_tu_thuc_hien", "cach_thuc_thuc_hien",
    "co_quan_thuc_hien", "thu_tuc_lien_quan"
]

FIELD_VN_NAME = {
    "ten_thu_tuc": "Tên thủ tục",
    "yeu_cau_dieu_kien": "Yêu cầu, điều kiện",
    "thanh_phan_ho_so": "Thành phần hồ sơ",
    "trinh_tu_thuc_hien": "Trình tự thực hiện",
    "cach_thuc_thuc_hien": "Cách thức thực hiện",
    "co_quan_thuc_hien": "Cơ quan thực hiện",
    "thu_tuc_lien_quan": "Thủ tục liên quan"
}

# Biến global cho model semantic (lazy load)
SEMANTIC_MODEL = None
SEMANTIC_MODEL_NAME = "all-MiniLM-L6-v2"  # có thể đổi theo nhu cầu

def normalize_text(s: str) -> str:
    """Chuẩn hóa text"""
    if not s:
        return ""
    s = str(s).replace("\r\n", "\n").replace("\t", " ").strip()
    s = re.sub(r"[ \u00A0]+", " ", s)
    s = re.sub(r"\n{3,}", "\n\n", s)
    return s

def split_text_safe(text: str, max_chars: int = 800) -> Generator[str, None, None]:
    """Chia text an toàn (fallback nếu không dùng semantic)"""
    text = normalize_text(text)
    if not text:
        return

    if len(text) <= max_chars:
        yield text
        return

    start = 0
    while start < len(text):
        end = min(len(text), start + max_chars)

        if end < len(text):
            cut = text.rfind(". ", start, end)
            if cut == -1 or cut <= start + 100:
                cut = text.rfind("\n", start, end)
            if cut == -1 or cut <= start + 50:
                cut = text.rfind(", ", start, end)
            if cut == -1 or cut <= start + 50:
                cut = end
        else:
            cut = end

        piece = text[start:cut].strip()
        if piece:
            yield piece

        if cut == end and end < len(text):
            start = cut
        else:
            start = cut + 1

        if start >= len(text):
            break

def load_semantic_model(model_name: str = SEMANTIC_MODEL_NAME):
    """Lazy-load SentenceTransformer model nếu khả dụng"""
    global SEMANTIC_MODEL
    if SEMANTIC_MODEL is not None:
        return SEMANTIC_MODEL
    try:
        from sentence_transformers import SentenceTransformer
    except Exception as e:
        raise RuntimeError(f"Không có package sentence_transformers: {e}")

    try:
        SEMANTIC_MODEL = SentenceTransformer(model_name, device="cuda")

        # Ép dùng 2 threads cho chắc chắn
        if torch is not None:
            torch.set_num_threads(2)
        return SEMANTIC_MODEL
    except Exception as e:
        raise RuntimeError(f"Không thể load model '{model_name}': {e}")

def semantic_chunk_text(text: str, model=None, percentile_threshold: float = 0.9, min_sentence_len: int = 10) -> List[str]:
    """
    Chia văn bản thành các chunks dựa trên sự tương đồng về ngữ nghĩa.
    Nếu model=None, sẽ cố gắng load model global.
    Trả về danh sách chunks (mỗi chunk là chuỗi văn bản).
    """
    text = normalize_text(text)
    if not text:
        return []

    # Tách văn bản thành câu/layers nhỏ hơn: sử dụng xuống dòng + dấu câu
    # Giữ những câu đủ dài, ghép các câu quá ngắn vào câu trước/sau
    # Sử dụng regex để chia câu cơ bản (vẫn an toàn cho tiếng Việt)
    raw_sentences = re.split(r'(?<=[\.\?\!]\s)|\n+', text)
    sentences = [s.strip() for s in raw_sentences if s.strip()]
    if not sentences:
        return []

    # Nếu số câu quá lớn, có thể hợp nhóm bằng kích thước tối đa trước khi embedding
    # Tuy nhiên ở đây ta tạo embedding cho từng "sentence"
    try:
        if model is None:
            model = load_semantic_model()
    except Exception as e:
        # Nếu không load được model, fallback trả về split_text_safe
        print(f"⚠️ Semantic model không khả dụng: {e} — fallback sang split_text_safe")
        return list(split_text_safe(text))

    # Tạo embedding cho mỗi câu (batch để tránh tràn RAM)
    try:
        # convert_to_numpy giúp tiết kiệm so với tensor lớn nếu cần
        embeddings = model.encode(sentences, convert_to_numpy=True, show_progress_bar=False)
    except TypeError:
        # Một số phiên bản model không có convert_to_numpy arg
        embeddings = model.encode(sentences, show_progress_bar=False)
        embeddings = np.array(embeddings)

    if embeddings is None or len(embeddings) == 0:
        return sentences

    # Tính cosine similarity giữa các câu liền kề
    similarities = []
    for i in range(len(embeddings) - 1):
        a = embeddings[i]
        b = embeddings[i + 1]
        denom = (np.linalg.norm(a) * np.linalg.norm(b))
        if denom == 0:
            sim = 0.0
        else:
            sim = float(np.dot(a, b) / denom)
        similarities.append(sim)

    if not similarities:
        return sentences

    # Xác định ngưỡng ngắt dựa trên phần trăm
    try:
        breakpoint_threshold = np.percentile(similarities, int(percentile_threshold * 100))
    except Exception:
        breakpoint_threshold = sum(similarities) / len(similarities)

    # Tạo các chunks: ghép các câu liên tiếp nếu similarity >= threshold
    chunks = []
    current_chunk = sentences[0]
    for i in range(len(similarities)):
        # Nếu similarity nhỏ hơn threshold -> break
        if similarities[i] < breakpoint_threshold:
            chunks.append(current_chunk.strip())
            current_chunk = sentences[i + 1]
        else:
            current_chunk += " " + sentences[i + 1]
    chunks.append(current_chunk.strip())

    # Sau khi tạo chunks, đảm bảo mỗi chunk không quá dài (nếu quá dài, fallback cắt bằng split_text_safe)
    final_chunks = []
    MAX_CHARS = 1200  # ngưỡng mềm, có thể điều chỉnh
    for ch in chunks:
        if len(ch) > MAX_CHARS:
            # sử dụng split_text_safe để cắt các đoạn quá dài
            for p in split_text_safe(ch, max_chars=MAX_CHARS):
                final_chunks.append(p)
        else:
            final_chunks.append(ch)

    # Loại bỏ các chunk quá ngắn (kết hợp với chunk trước nếu cần)
    merged_chunks = []
    for ch in final_chunks:
        if merged_chunks and len(ch) < min_sentence_len:
            merged_chunks[-1] = (merged_chunks[-1] + " " + ch).strip()
        else:
            merged_chunks.append(ch)

    return merged_chunks

def create_chunk(record: Dict[str, Any], field: str, piece: str, piece_idx: int) -> str:
    """Tạo một chunk JSON"""
    parent_id = str(record.get("nguon", str(uuid.uuid4())[:8]))
    title = normalize_text(record.get("ten_thu_tuc", ""))
    source = str(record.get("nguon", ""))

    field_name = FIELD_VN_NAME.get(field, field)
    text_content = f"Thủ tục: {title}\nMục: {field_name}\nNội dung: {piece}"

    chunk = {
        "id": f"{parent_id}#{field}#{piece_idx}",
        "parent_id": parent_id,
        "ten_thu_tuc": title,
        "field": field,
        "text": text_content,
        "raw": piece,
        "nguon": source
    }

    return json.dumps(chunk, ensure_ascii=False)

def process_record_streaming(record: Dict[str, Any], outfile) -> int:
    """Xử lý một record và ghi trực tiếp ra file
    Sử dụng semantic chunking nếu model khả dụng, ngược lại fallback sang split_text_safe.
    Trả về số chunk tạo ra.
    """
    chunk_count = 0

    for field in FIELD_ORDER:
        raw_value = record.get(field)
        if not raw_value:
            continue

        text_value = str(raw_value).strip()
        if not text_value:
            continue

        # Thử semantic chunking trước
        try:
            pieces = semantic_chunk_text(text_value, model=None, percentile_threshold=0.9)
            if not pieces:
                # Nếu semantic trả về rỗng, fallback
                pieces = list(split_text_safe(text_value))
        except Exception as e:
            print(f"⚠️ Lỗi semantic chunking cho field {field}: {e}")
            pieces = list(split_text_safe(text_value))

        # Xử lý từng piece
        piece_idx = 0
        for piece in pieces:
            if not piece or not piece.strip():
                continue

            try:
                chunk_json = create_chunk(record, field, piece, piece_idx)
                outfile.write(chunk_json + "\n")
                chunk_count += 1
                piece_idx += 1
            except Exception as e:
                print(f"❌ Lỗi tạo chunk cho field {field}: {e}")
                continue

    return chunk_count

def safe_json_stream_parser(filepath: str) -> Generator[Dict[str, Any], None, None]:
    """Parser JSON streaming an toàn hơn"""

    print("🔄 Đang parse JSON stream...")

    try:
        with open(filepath, 'r', encoding='utf-8', buffering=8192) as f:
            # Đọc và kiểm tra ký tự đầu
            first_char = f.read(1)
            if first_char != '[':
                print(f"❌ File không bắt đầu bằng '[', mà là '{first_char}'")
                return

            # Reset về đầu và bỏ qua '['
            f.seek(1)

            buffer = ""
            brace_count = 0
            bracket_count = 0
            in_string = False
            escape_next = False
            char_count = 0

            while True:
                char = f.read(1)
                if not char:
                    break

                char_count += 1

                # Xử lý escape characters
                if escape_next:
                    buffer += char
                    escape_next = False
                    continue

                if char == '\\' and in_string:
                    buffer += char
                    escape_next = True
                    continue

                # Xử lý strings
                if char == '"':
                    in_string = not in_string
                    buffer += char
                    continue

                if in_string:
                    buffer += char
                    continue

                # Xử lý cấu trúc JSON
                if char == '{':
                    brace_count += 1
                    buffer += char
                elif char == '}':
                    brace_count -= 1
                    buffer += char

                    # Hoàn thành một object
                    if brace_count == 0 and bracket_count == 0:
                        try:
                            # Clean buffer trước khi parse
                            clean_buffer = buffer.strip().rstrip(',')
                            if clean_buffer:
                                obj = json.loads(clean_buffer)
                                yield obj
                                del obj
                        except json.JSONDecodeError as e:
                            print(f"❌ JSON decode error tại ký tự {char_count}: {e}")
                            print(f"   Buffer: {buffer[:100]}...")

                        # Reset buffer
                        buffer = ""
                        gc.collect()

                elif char == '[':
                    bracket_count += 1
                    buffer += char
                elif char == ']':
                    if bracket_count > 0:
                        bracket_count -= 1
                        buffer += char
                    else:
                        # Kết thúc main array
                        break
                elif char == ',':
                    if brace_count == 0 and bracket_count == 0:
                        # Dấu phẩy giữa các objects chính
                        continue
                    else:
                        buffer += char
                elif char in ' \t\n\r':
                    # Whitespace
                    if buffer.strip():  # Chỉ thêm nếu buffer không rỗng
                        buffer += ' '
                else:
                    buffer += char

                # Progress indicator
                if char_count % 100000 == 0:
                    print(f"📖 Đã đọc {char_count} ký tự...")

    except Exception as e:
        print(f"❌ Lỗi đọc file: {e}")

def chunking_main():
    """Hàm chunking chính"""

    print("=== BẮT ĐẦU CHUNKING (semantic-enabled) ===")

    if not os.path.exists(RAW_JSON_PATH):
        print(f"❌ File không tồn tại: {RAW_JSON_PATH}")
        return

    file_size_mb = os.path.getsize(RAW_JSON_PATH) / (1024 * 1024)
    print(f"📁 Kích thước file: {file_size_mb:.2f} MB")

    total_chunks = 0
    processed_records = 0

    try:
        with open(CHUNKS_JSONL, 'w', encoding='utf-8', buffering=1024) as outfile:

            for record in safe_json_stream_parser(RAW_JSON_PATH):
                try:
                    chunks_created = process_record_streaming(record, outfile)
                    total_chunks += chunks_created
                    processed_records += 1

                    # Progress report
                    if processed_records % 50 == 0:
                        print(f"✅ Đã xử lý: {processed_records} records → {total_chunks} chunks")
                        outfile.flush()
                        gc.collect()

                except Exception as e:
                    print(f"❌ Lỗi xử lý record {processed_records}: {e}")
                    continue

                # Cleanup
                del record

    except Exception as e:
        print(f"❌ Lỗi fatal: {e}")
        return

    print(f"\n🎉 HOÀN THÀNH!")
    print(f"📊 Tổng kết: {processed_records} records → {total_chunks} chunks")
    print(f"💾 Output: {CHUNKS_JSONL}")

    # Kiểm tra file output
    if os.path.exists(CHUNKS_JSONL):
        output_size = os.path.getsize(CHUNKS_JSONL) / (1024 * 1024)
        print(f"📁 Kích thước output: {output_size:.2f} MB")

def fallback_simple_load():
    """Phương án dự phòng: load trực tiếp (cho file nhỏ)"""
    print("🔄 Thử phương án load trực tiếp...")

    try:
        with open(RAW_JSON_PATH, 'r', encoding='utf-8') as f:
            data = json.load(f)

        print(f"✅ Load thành công! Tổng records: {len(data)}")

        total_chunks = 0

        with open(CHUNKS_JSONL, 'w', encoding='utf-8') as outfile:
            for i, record in enumerate(data):
                try:
                    chunks_created = process_record_streaming(record, outfile)
                    total_chunks += chunks_created

                    if (i + 1) % 100 == 0:
                        print(f"✅ {i + 1}/{len(data)} → {total_chunks} chunks")
                        outfile.flush()
                        gc.collect()

                except Exception as e:
                    print(f"❌ Lỗi record {i}: {e}")
                    continue

        print(f"🎉 Fallback hoàn thành: {len(data)} records → {total_chunks} chunks")

    except Exception as e:
        print(f"❌ Fallback thất bại: {e}")

def main():
    """Main function"""
    print("=== ROBUST JSON CHUNKING (semantic-enabled) ===")

    try:
        chunking_main()
    except Exception as e:
        print(f"❌ Phương pháp chính thất bại: {e}")
        print("🔄 Thử phương án dự phong...")
        fallback_simple_load()

if __name__ == "__main__":
    main()


=== ROBUST JSON CHUNKING (semantic-enabled) ===
=== BẮT ĐẦU CHUNKING (semantic-enabled) ===
📁 Kích thước file: 70.00 MB
🔄 Đang parse JSON stream...
✅ Đã xử lý: 50 records → 890 chunks
✅ Đã xử lý: 100 records → 1835 chunks
✅ Đã xử lý: 150 records → 2616 chunks
✅ Đã xử lý: 200 records → 3773 chunks
✅ Đã xử lý: 250 records → 4758 chunks
✅ Đã xử lý: 300 records → 5651 chunks
✅ Đã xử lý: 350 records → 6623 chunks
✅ Đã xử lý: 400 records → 7590 chunks
✅ Đã xử lý: 450 records → 8651 chunks
✅ Đã xử lý: 500 records → 10168 chunks
✅ Đã xử lý: 550 records → 11190 chunks
✅ Đã xử lý: 600 records → 12225 chunks
✅ Đã xử lý: 650 records → 13348 chunks
✅ Đã xử lý: 700 records → 14343 chunks
✅ Đã xử lý: 750 records → 15307 chunks
✅ Đã xử lý: 800 records → 16405 chunks
✅ Đã xử lý: 850 records → 17402 chunks
✅ Đã xử lý: 900 records → 18411 chunks
✅ Đã xử lý: 950 records → 19359 chunks
✅ Đã xử lý: 1000 records → 20278 chunks
✅ Đã xử lý: 1050 records → 21386 chunks
✅ Đã xử lý: 1100 records → 22712 chunks
✅ 

In [5]:
# =========================
# 4) XÂY DỰNG EMBEDDINGS + FAISS
# =========================
from tqdm.auto import tqdm

emb_model = SentenceTransformer(EMBED_MODEL_NAME, device="cuda")
emb_model = emb_model.half()

texts, metas = [], []
with open(CHUNKS_JSONL, "r", encoding="utf-8") as f:
    for line in f:
        obj = json.loads(line)
        texts.append(obj["text"])
        metas.append(obj)

emb = emb_model.encode(
    texts,
    batch_size=64,
    show_progress_bar=True,
    normalize_embeddings=True
).astype("float32")

index = faiss.IndexFlatIP(emb.shape[1])  # cosine khi đã normalize
index.add(emb)

faiss.write_index(index, FAISS_INDEX)
with gzip.open(METAS_PKL_GZ,"wb") as g:
    pickle.dump(metas, g)

print("Đã build FAISS:", FAISS_INDEX, "| metas:", METAS_PKL_GZ)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/171 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

Batches:   0%|          | 0/4162 [00:00<?, ?it/s]

Đã build FAISS: /content/drive/MyDrive/eGov-Bot/final_RAG/index.faiss | metas: /content/drive/MyDrive/eGov-Bot/final_RAG/metas.pkl.gz


In [6]:
# =========================
# 5) HYBRID RETRIEVAL: BM25
# =========================
# Dùng raw text (không cần tiền tố) để ưu tiên khớp từ khoá pháp lý.
bm25_corpus = [m["raw"] for m in metas]
bm25 = BM25Okapi([t.split() for t in bm25_corpus])
with gzip.open(BM25_PICKLE, "wb") as g:
    pickle.dump(bm25, g)
print("Đã khởi tạo BM25.")

Đã khởi tạo BM25.
